In [46]:
## Load MNIST
## Loads MNIST Dataset from torchvision and creates train, valid, test dataloaders.
## PyTorch
import torch
from torchvision import datasets
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import ToTensor

SPLIT = 0.8
BATCH_SIZE = 64

train_dataset = datasets.MNIST("data", train=True, download=True, transform=ToTensor())
test_dataset = datasets.MNIST("data", train=False, download=True, transform=ToTensor())
train_dataset, valid_dataset = random_split(train_dataset, [int(SPLIT*len(train_dataset)), len(train_dataset)-int(SPLIT*len(train_dataset))], generator=torch.Generator().manual_seed(42))

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False) # Note that shuffle is turned off for the test set

print(f"> Data Loaded: \n  {len(train_dataset):,} train\n  {len(valid_dataset):,} valid\n  {len(test_dataset):,} test")


> Data Loaded: 
  48,000 train
  12,000 valid
  10,000 test


In [ ]:
## Show image batch
## Shows a batch of images from pytorch dataloaders
## PyTorch, Matplotlib, Numpy
import matplotlib.pyplot as plt
import numpy as np

def show_image_batch(dataloader, n_rows=8, figsize=(10,10)):
    images, labels = next(iter(dataloader))
    N = len(images)
    fig, axs = plt.subplots(n_rows, N//n_rows, figsize=(10,10), constrained_layout=True)
    
    for image, label, ax in zip(images, labels, axs.ravel()[:N]):
        ax.set_title(label.item())
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(image.squeeze())

show_image_batch(train_dataloader, n_rows=8)

In [52]:
## Set Device
## Set the PyTorch device either to GPU or CPU
## PyTorch
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {DEVICE} device")

Using cpu device


In [ ]:
## Custom Dataset
## The boilerplate code for creating a custom PyTorch Datast.
## PyTorch, Pandas
import os
import pandas as pd
from torch.utils.data import Dataset
from torchvision.io import read_image

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
## Simple Neural Network
## Simple PyTorch NeuralNetwork Module
## PyTorch
import torch.nn as nn
from collections import OrderedDict

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            OrderedDict([
                ("linear1", nn.Linear(28*28, 512)),
                ("relu1", nn.ReLU()),
                ("linear2", nn.Linear(512, 512)),
                ("relu2", nn.ReLU()),
                ("linear3", nn.Linear(512, 10)),
            ])
            
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
## LeNet
## Create the LeNet Architecture in PyTorch
## PyTorch
import torch.nn as nn
from collections import OrderedDict

model = nn.Sequential(OrderedDict([
    # First Convolution Block
    ("conv1", nn.Conv2d(1, 6, 5, padding=2)),
    ("relu1", nn.ReLU()),
    ("avgpool1", nn.AvgPool2d(2, stride=2)),
    
    # Second Convolution Block
    ("conv2", nn.Conv2d(6, 16, 5, padding=2)),
    ("relu2", nn.ReLU()),
    ("avgpool2", nn.AvgPool2d(2, stride=2)),
    
    # Classifier
    ("flatten", nn.Flatten()),
    ("linear1", nn.Linear(784, 120)),
    ("relu3", nn.ReLU()),
    ("linear2", nn.Linear(120, 84)),
    ("relu4", nn.ReLU()),
    ("linear3", nn.Linear(84, 10)),
]))

In [ ]:
## Loss function
## Quickly create a PyTorch loss function
## PyTorch
import torch.nn as nn

loss_fn = nn.CrossEntropyLoss()

In [53]:
## Optimizer
## Quickly create a PyTorch optimizer
## PyTorch
import torch

LEARNING_RATE = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

NameError: name 'model' is not defined

In [ ]:
## Simple Training loop
## A simple training loop for PyTorch
## PyTorch

def train_model(model, dataloader, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [78]:
a = time.perf_counter()
time.sleep(2)
b = time.perf_counter()
b-a

2.0057309999974677

In [79]:
## Complete Training loop
## A complete training loop with progressbar and loss history.
## PyTorch
from tqdm.notebook import tqdm, trange
import copy
import time

def train_step(model, dataloader, loss_fn, optimizer, current_epoch, num_epochs):
    
    # tqdm progressbar
    pbar = tqdm(total=len(dataloader.dataset), desc=f"Epoch {current_epoch}/{num_epochs} ")
    
    # For every batch
    for batch, (X, y) in enumerate(dataloader):
        
        # Compute prediction and loss
        pred = model(X)
        train_loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        # Calculate losses
        train_loss = train_loss.item()
        valid_loss = train_loss # TODO: calculate validation loss
        
        # Update tqdm progressbar
        pbar.set_postfix({"Train Loss":f"{train_loss:.6f}", "Valid Loss":f"{valid_loss:.6f}"})
        pbar.update(len(X))
        
    return train_loss, valid_loss

def train_model(model, train_dataloader, loss_fn, optimizer, epochs, weights_path="model.pt"):
    # Track time
    start_time = time.perf_counter()
    
    # Best model and loss
    best_valid_loss = float('inf')
    best_model = model
    
    # Loss histories for plotting
    train_loss_history = []
    valid_loss_history = []
    
    # For every epoch
    for i in range(epochs):
        # Train step
        train_loss, valid_loss = train_step(model, train_dataloader, loss_fn, optimizer, current_epoch=i+1, num_epochs=epochs)
        
        # Track losses
        train_loss_history.append(train_loss)
        valid_loss_history.append(valid_loss)
        
        # Save best model
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            best_model = copy.deepcopy(model)
            torch.save(best_model.state_dict(), weights_path)
            tqdm.write(f"> Best Model Saved: {weights_path}")
    
    print(f"Training Done in {time.perf_counter() - start_time} seconds.")
    print(f"best Validation Loss: {best_valid_loss:.6f}")
    
    return best_model, train_loss_history, valid_loss_history

best_model, best_loss = train_model(model, train_dataloader, loss_fn, optimizer, epochs=4, weights_path="model.pt")


NameError: name 'model' is not defined

In [1]:
## Plot loss
## Plot the training and validation loss history returned by the train_model function
## PyTorch, Matplotlib

def plot_loss(train_loss_history, valid_loss_history):
    plt.figure()
    plt.title("Loss")
    plt.xlabel("epoch")
    plt.ylabel("Loss")
    
    plt.plot(np.arange(len(train_loss_history)), train_loss_history)
    plt.plot(np.arange(len(valid_loss_history)), valid_loss_history)
    
    plt.legend(["Train Loss","Valid Loss"])
    plt.show()

plot_loss(train_loss_history, valid_loss_history)

NameError: name 'train_loss_history' is not defined

In [ ]:
model = Neural_Network()
model.to(DEVICE)
model.load_state_dict(torch.load('/content/best-weights.pt'))

In [ ]:
## Torchvision transforms
## A set of torchvision transforms ready to be passed to a Dataset
## PyTorch

from torchvision import transforms as T 

train_augs = T.Compose([
    T.RandomHorizontalFlip(p = 0.5),
    T.RandomRotation(degrees=(-20, +20)),
    T.ToTensor()

])

valid_augs = T.Compose([
    T.ToTensor()
])

In [ ]:
## Load datasets using ImageFolder
## Load train and valid image dataset when you have folders of images for each class
## PyTorch
from torchvision.datasets import ImageFolder

trainset = ImageFolder("data/train", transform=train_augs)
validset = ImageFolder("data/valid", transform=valid_augs)

In [ ]:
## PyTorch Multi Transform
## transforms for train and valid Datasets, provided as a dictionary
## PyTorch
data_transforms = {
    'train': T.Compose([
        T.RandomResizedCrop(INPUT_SIZE),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': T.Compose([
        T.Resize(INPUT_SIZE),
        T.CenterCrop(INPUT_SIZE),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
## Download data if not exists
## Downloads the data using wget only if that data hasn't been downloaded yet
## Other
!wget -nc https://path.com/to/file

In [ ]:
## Unzip data if not exists
## Unzips the data using unzip tool only if that data hasn't been extracted yet
## Other
!unzip -n "/content/A.zip" -d "/content/A/"

In [ ]:
## Unrar data if not exists
## Unrars the data using unrar tool only if that data hasn't been extracted yet
## Other
!unrar -o -x "/content/A.rar" "/content/A/"

In [ ]:
## Show GPU info
## Prints info about your gpu. useful in google colab
## Other
!nvidia-smi

In [ ]:
## Show RAM info
## Prints info about your RAM.
## Other
!cat /proc/meminfo

In [ ]:
## Profiling (%prun)
## Sample code and usage of profiler in jupyter notebooks
## Other
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
    return total

%prun sum_of_lists(1000000)

In [ ]:
## Line Profiler (%lprun)
## Installing and usage example of line_profiler
## Other
!pip install line_profiler

%load_ext line_profiler

%lprun -f sum_of_lists sum_of_lists(5000)

In [ ]:
## Memory Profiler (%memit)
## Installing and usage example of memory_profiler
## Other
!pip install memory_profiler

%load_ext memory_profiler

%memit sum_of_lists(1000000)

In [ ]:
## Dataset Loading and Preperation
## Loads a dataset, shuffles it, splits it into train and test set and seperates label from features
## Pandas
# Load data
df = pd.read_csv("/content/heart.csv")

# Shuffle dataframe
df = df.sample(frac=1.0).reset_index(drop=True)

# Seperate X,y
X = df.drop(columns=["target"])
y = df["target"]

# Split to train and test
split = 0.7

X_train = X.iloc[ : int(len(X)*split),:].reset_index(drop=True)
X_test = X.iloc[int(len(X)*split) : ,:].reset_index(drop=True)

y_train = y.iloc[ : int(len(X)*split)].reset_index(drop=True)
y_test = y.iloc[int(len(X)*split) : ].reset_index(drop=True)

print(f"Train X size = {len(X_train)}")
print(f"Train y size = {len(y_train)}")
print(f"Test X size = {len(X_test)}")
print(f"Test y size = {len(y_test)}")

In [ ]:
## Pandas Complete read_csv
## read_csv but with custom sperator and header names
## Pandas
# Custom 'sep', 'Nan symbol' and 'header_names'
df = pd.read_csv('crx.data', header=None, sep=",", na_values=["?"], names=["A1", "A2", "A3", "A4", "A5", "A6", "A7", "A8", "A9", "A10", "A11", "A12", "A13", "A14", "A15", "A16"])

In [ ]:
## Find Columns with Missing Values
## What columns of a pandas DataFrame have missing values and how many?
## Pandas
df.isna().sum()

In [ ]:
## One-hot encode Categorical Data
## One-hot encode categorical data pandas DataFrame
## Pandas
for col in train_df.dtypes[train_df.dtypes == 'object'].index:
    for_dummy = train_df.pop(col)
    train_df = pd.concat([train_df, pd.get_dummies(for_dummy, prefix=col)], axis=1)

In [ ]:
## Matplotlib customizations
## 
## Matplotlib
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
sns.set_style("dark") # to disable gridlines

# Subplots
fig, axs = plt.subplots(5, 4, figsize=(20,10), dpi=100)

# Disable Ticks
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)


In [ ]:
## Matplotlib disable ticks
## Matplotlib disable ticks
## Matplotlib
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)

In [ ]:
## Matplotlib text
## Matplotlib text customizations
## Matplotlib
ax.text(0.5,
        -0.09,
        "Hiiii",
        transform=ax.transAxes,
        c="r",
        horizontalalignment='center',
        verticalalignment='center',
        fontsize=12)

In [ ]:
## Plot confusion matrix
## Plot confusion matrix using sk-learn's 'ConfusionMatrixDisplay'
## Matplotlib, sk-learn
from sklearn.metrics import ConfusionMatrixDisplay

disp = ConfusionMatrixDisplay(confusion_matrix=PASS_HERE, display_labels=labels)
disp.plot(cmap="gray")
plt.show()

In [ ]:
## Plot histogram of pandas columns
## Plot histogram of pandas columns
## Pandas
# Continuous Feature
sns.histplot(data=df.column_name, bins=30, kde=True, color='green')

# Categorical Feature
sns.histplot(data=df.column_name)
df.column_name.value_counts()

In [ ]:
## Tensorboard, Early Stopping and Epoch Dots
## 
## Tensorflow
!pip install git+https://github.com/tensorflow/docs

import tensorflow_docs as tfdocs

import pathlib
import shutil
import tempfile

logdir = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
shutil.rmtree(logdir, ignore_errors=True)

# pass this callback to the 'fit' function
def get_callbacks(name):
  return [
		# useful for trainings with a lot of epochs prints "." instead of every epoch
    tfdocs.modeling.EpochDots(),
		# performs early stopping 
    tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=200),
		# saves data for tensorboard
    tf.keras.callbacks.TensorBoard(logdir/name),
  ]

# ANDDDD
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Open an embedded TensorBoard viewer
%tensorboard --logdir {logdir}

# ORRRRR
# upload to tensorboard.dev 
tensorboard dev upload --logdir  {logdir}

# and add an IFrame into the notebook
display.IFrame(
    src="https://tensorboard.dev/experiment/vW7jmmF9TmKmy3rbheMQpw/#scalars&_smoothingWeight=0.97",
    width="100%", height="800px")

In [ ]:
## Fourier Analysis
## Quickly display the magnitude and phase of an image using fourier analysis
## OpenCV
def fourier_analysis(img):
    fourier_img = cv2.dft(img.astype(np.float32), flags=cv2.DFT_COMPLEX_OUTPUT)
    fourier_img_shift = np.fft.fftshift(fourier_img)
    real = fourier_img_shift[:,:,0]
    imag = fourier_img_shift[:,:,1]
    magnitude = cv2.magnitude(real, imag)
    phase = cv2.phase(real, imag)
    return magnitude, phase

In [ ]:
## Apply Fourier Filter
## Applies fourier filter to an image given the mask and returns the filtered image
## OpenCV
def apply_fourier_filter(img, mask):
    mask = mask[:, :, np.newaxis]
    img_fourier = cv2.dft(img.astype(np.float32), flags=cv2.DFT_COMPLEX_OUTPUT)
    img_fourier_shift = np.fft.fftshift(img_fourier)
    img_fourier_shift *= mask
    img_fourier_shift_back = np.fft.ifftshift(img_fourier_shift)
    img_fourier_inverse = cv2.idft(img_fourier_shift_back, flags=cv2.DFT_SCALE)

    return img_fourier_inverse

In [ ]:
## GHD OpenCV utilities
## simple functions like disp, rgb, bgr, uint8ify for quickly prototyping opencv code
## OpenCV
def rgb(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def bgr(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

def gray(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def disp(img, title='', s=8, vmin=None, vmax=None):
    plt.figure(figsize=(s,s))
    plt.axis('off')
    if vmin is not None and vmax is not None:
        plt.imshow(img, cmap='gray', vmin=vmin, vmax=vmax)
    else:
        plt.imshow(img, cmap='gray')
    plt.title(title)
    plt.show()

def uint8ify(img):
    img -= img.min()
    img /= img.max()
    img *= 255
    return np.uint8(img)

def overlay(a,b):
    # a and b should be float images and between 0-1

    mask = a >= 0.5 # generate boolean mask of everywhere a > 0.5 
    ab = np.zeros_like(a) # generate an output container for the blended image 

    # now do the blending 
    ab[~mask] = (2*a*b)[~mask] # 2ab everywhere a<0.5
    ab[mask] = (1-2*(1-a)*(1-b))[mask] # else this
    
    return ab

def before_after(img_a, img_b, name='', vmin=None, vmax=None, effect_name='Processed'):
    fig, axs = plt.subplots(1,2, constrained_layout=True, figsize=(10,4))
    axs[0].axis('off')
    axs[1].axis('off')
    axs[0].set_title(f'{name} Original')
    axs[1].set_title(f'{name} {effect_name}')
    if vmin is not None and vmax is not None:
        axs[0].imshow(img_a, cmap='gray', vmin=vmin, vmax=vmax)
        axs[1].imshow(img_b, cmap='gray', vmin=vmin, vmax=vmax)
    else:
        axs[0].imshow(img_a, cmap='gray')
        axs[1].imshow(img_b, cmap='gray')
    plt.show()